In [6]:
import struct
import time
import json
import os
import math

SHS_DEGREE = 0
ENDIAN = "!"

READ_FILE = "cs.dat"

READ_FORMAT = {
    'start_frame': 'I',
    'end_frame': 'I',
    'xyz': 'fff',
    'f_dc': 'fff',
    'f_rest': 'fff' * ((SHS_DEGREE + 1) ** 2 -1),
    'scaling': 'fff',
    'rotation': 'ffff',
    'opacity': 'f'
}

WRITE_FORMAT = {
    'start_frame': 'H',
    'end_frame': 'H',
    'xyz': 'fff',
    'color': 'BBB',
    'opacity': 'B',
    'scaling': 'fff',
    'rotation': 'BBBB'
}


In [7]:

def stream_dump(params, filename, format):
    '''
    name: fromF, toF, xyz, feature, s,   r,    o
    data: I    , I,   fff, fffx4,   fff, ffff, f
    '''
    FORMAT = format
    N = len(params)

    fmt = f"{ENDIAN}{''.join(FORMAT.values())}"
    print(f"Format: {fmt}, total bytes: {struct.calcsize(fmt)}")

    dir = os.path.dirname(filename)
    with open(os.path.join(dir, 'format.json'), 'w') as f:
        json.dump(FORMAT, f, indent=4)

    time_start = time.time()
    values = []
    for v in params:
        values.append(struct.pack(fmt, *v))
    with open(filename, 'ab') as f:
        f.writelines(values)
    time_end = time.time()

    print(f"Dumped {N} gaussians in {time_end - time_start} seconds")

def stream_load(filename, format):
    FORMAT = format
    fmt = f"{ENDIAN}{''.join(FORMAT.values())}"
    print(f"Format: {fmt} total bytes: {struct.calcsize(fmt)}")
    with open(filename, 'rb') as f:
        data = f.read()
    N = len(data) // struct.calcsize(fmt)
    print(f"Loading {N} gaussians")
    unpacked = []
    for i in range(N):
        unpacked.append(
            struct.unpack(fmt,
                          data[i * struct.calcsize(fmt): (i+1) * struct.calcsize(fmt)]))
    return unpacked

In [8]:
def rgba(f_dc, opacity):
    clamp = lambda x: min(255, max(0, x))
    SH_C0 = 0.28209479177387814
    color = [0, 0, 0]
    color[0] = clamp(int((0.5 + SH_C0 * f_dc[0]) * 255))
    color[1] = clamp(int((0.5 + SH_C0 * f_dc[1]) * 255))
    color[2] = clamp(int((0.5 + SH_C0 * f_dc[2]) * 255))
    alpha = int((1 / (1 + math.exp(-opacity))) * 255)
    return (*color, alpha)
def rot(rotation):
    qlen = math.sqrt(
            rotation[0] ** 2 +
            rotation[1] ** 2 +
            rotation[2] ** 2 +
            rotation[3] ** 2,
        )
    rot = [0, 0, 0, 0]
    rot[0] = int((rotation[0] / qlen) * 128 + 128)
    rot[1] = int((rotation[1] / qlen) * 128 + 128)
    rot[2] = int((rotation[2] / qlen) * 128 + 128)
    rot[3] = int((rotation[3] / qlen) * 128 + 128)
    return rot

def scale(scaling):
    scales = [0, 0, 0]
    scales[0] = math.exp(scaling[0])
    scales[1] = math.exp(scaling[1])
    scales[2] = math.exp(scaling[2])
    return scales

In [ ]:

ret = stream_load(READ_FILE, READ_FORMAT)
print(ret[0])

shrank = []
for r in ret:
    start_frame = r[0]
    end_frame = r[1]
    xyz = r[2:5]
    f_dc = r[5:8]
    shs_length = 3 * ((SHS_DEGREE + 1) ** 2 - 1)
    f_rest = r[8:8 + shs_length]
    scaling = r[8 + shs_length: 11 + shs_length]
    rotation = r[11 + shs_length: 15 + shs_length]
    opacity = r[-1]
    shrank.append(
        (
            start_frame,
            end_frame,
            *xyz,
            *(rgba(f_dc, opacity)),
            *(scale(scaling)),
            *(rot(rotation)),
        )
    )



In [ ]:


new_name = f"{READ_FILE[:-4]}_36B.dat"
stream_dump(shrank, new_name, WRITE_FORMAT)
# test = stream_load(new_name, WRITE_FORMAT)
# print(test[0])